# Neighbourhood-Based Performance Prediction on MNIST

## 0. Preparation 
### Load Test Data: Embedding, Softmax Activations and Labels

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [29]:
embeddings = pd.read_csv('test_activations.csv', usecols = [i for i in range(1, 129)])
labels = pd.read_csv('test_labels.csv', usecols = [1])
log_softmax = pd.read_csv('test_log_softmax.csv', usecols = [i for i in range(1, 11)])

In [3]:
# transform log_softmax to softmax
softmax = np.round(np.exp(log_softmax), decimals=5) # numerical irregularities

### Load embeddings of Training Data

In [36]:
train_embeddings = pd.read_csv('train_activations.csv', usecols = [i for i in range(1, 129)])
train_labels = pd.read_csv('train_labels.csv', usecols = [1])
train_log_softmax = pd.read_csv('train_log_softmax.csv', usecols = [i for i in range(1, 11)])

In [42]:
train_softmax = np.round(np.exp(train_log_softmax), decimals=5) # numerical irregularities

## 1. Ground Truth and Baseline
### Compute true Accuracy

In [4]:
predictions = softmax.idxmax(axis=1)
predictions = predictions.astype('int')

In [5]:
true_acc = (predictions == pd.Series(labels['0'])).sum() / len(predictions)
print('True test-set accuracy of the model: ', true_acc)

True test-set accuracy of the model:  0.9865


### Estimate Accuracy through average softmax confidence

In [51]:
conf_acc = softmax.max(axis=1).mean()
print('Average softmax confidence: ', conf_acc)

Average softmax confidence:  0.989314257000017


## 2. Nearest Neighbor Approach

### Find neighboring training points for each test point

In [79]:
k = 10
# For each test point: Find nearest training neighbours in embedding space
nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(train_embeddings)
nb_distances, nb_indices = nbrs.kneighbors(embeddings)

### Average softmax-confidence of all neighbors

In [80]:
# Average confidences over neighbors
neighbor_softmax = []
n_obs = nb_indices.shape[0]
for i in range(n_obs):
    neighbor_confs = train_softmax.max(axis=1)[nb_indices[i, :]].mean()
    neighbor_softmax.append(neighbor_confs)

In [81]:
np.array(neighbor_softmax).mean()

0.9936490487000001

<b> 1. Neighborhood Confidence performs worse. </b>

But: We did not include the information on whether the prediction was correct.

$\leftarrow$ Rationale:  When a close neighbor's prediction was wrong despite high confidence, we shall decrease the confidence of the query point.

$\leftarrow$ Check this heuristic visually: For all misclassified test points plot the fraction of wrong predictions in the neighborhood.

<b> 2. We seem to identify appropriate nearest neighbors, as our estimation is close to GT. </b>

### Multiply confidence with fraction of correct classifications in the neighborhood